# MATH3976 Project: SimpleCircuits.jl

David Rapisarda

SID: 440429320

Note: this has two dependencies - [DataStructures.jl](https://github.com/JuliaLang/DataStructures.jl) and [Sundials.jl](https://github.com/JuliaLang/Sundials.jl). 

In [1]:
include("SimpleCircuits.jl")
using SimpleCircuits
using PyPlot
versioninfo()

Julia Version 0.4.5
Commit 2ac304d (2016-03-18 00:58 UTC)
Platform Info:
  System: Linux (x86_64-linux-gnu)
  CPU: Intel(R) Core(TM) i5-4670 CPU @ 3.40GHz
  WORD_SIZE: 64
  BLAS: libopenblas (NO_LAPACK NO_LAPACKE DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: liblapack.so.3
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


This document is split into three parts: in the first, the implementation details of the simulator are covered. In the second part, some examples of the simulator in use are shown. In the last part, the originality of the project in the context of existing research is considered, and some avenues for further work are discussed.

## Part 1: Implementation Details

The 

## Part 2: Examples

### Example 1: Voltage Divider

<img src="img/example1.png" />

This is the simplest example of any interest: two resistors connected in series, excited by a DC voltage source. 

In [2]:
circ = Circuit()
r1 = Resistor(5e+3)
r2 = Resistor(10e+3)
vin = DCVoltageSource(5.)
connect!(circ, vin.pLow, circ.gnd)
connect!(circ, vin.pHigh, r1.p1)
connect!(circ, r1.p2, r2.p1, "Vout")
connect!(circ, r2.p2, circ.gnd)

cop = op(circ)

LoadError: LoadError: MethodError: `newton` has no method matching newton(::Function, ::Function, ::Array{Float64,1}, ::Dict{Symbol,Float64})
Closest candidates are:
  newton(::Function, ::Function, ::Array{Float64,1})
  newton(::Function, ::Function, ::Array{Float64,1}, !Matched::Dict{Symbol,Union{Array{Float64,1},Float64}})
while loading In[2], in expression starting on line 10

See what happens when varing the value of R1:

In [3]:
# change the resistance of r1 to a parameter
r1.R = :R
params = Parameters(:R => 1.)
R_range = logspace(1., 6., 100)

# compute the operating point for all values of this parameter
cop = dc_sweep(circ, :R, R_range, params)

grid(b=true, which="major")
title("Voltage divider output vs R1")
xlabel("R1 (Ω)")
ylabel("Vout (V)")
plot(R_range, cop[r1.p2]);

LoadError: LoadError: MethodError: `dc_sweep` has no method matching dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Array{Float64,1}, ::Dict{Symbol,Union{Array{Float64,1},Float64}})
Closest candidates are:
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any)
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any, !Matched::Dict{Symbol,Float64})
while loading In[3], in expression starting on line 7

### Example 2: Simple diode circuit

<img src="img/example2.png" />

Linear circuit analysis is not so interesting, since Thevenin's theorem states that for any linear circuit there exists an equivalent circuit, with one voltage source and one resistor, which appears exactly the same as the original circuit, when looking at two nodes - so now let's examine a simple non-linear circuit. 

In [4]:
circ = Circuit()
r1 = Resistor(1e+3)
d1 = Diode() # using the default values for a diode

# we will be sweeping over the input voltage
vin = DCVoltageSource(:V)

connect!(circ, vin.pLow, circ.gnd)
connect!(circ, vin.pHigh, r1.p1)
connect!(circ, r1.p2, d1.pIn)
connect!(circ, d1.pOut, circ.gnd)

# default value for Vin - it will be overridden when we sweep anyway
params = Parameters(:V => 5.)

V_range = -5.:0.01:5.
cop = dc_sweep(circ, :V, V_range, params)

# plot the voltage on the diode
grid(b=true, which="major")
title("Voltage response of the diode circuit")
xlabel("Vin")
ylabel("Vout")
plot(V_range, cop[r1.p2]);

LoadError: LoadError: MethodError: `dc_sweep` has no method matching dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::FloatRange{Float64}, ::Dict{Symbol,Union{Array{Float64,1},Float64}})
Closest candidates are:
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any)
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any, !Matched::Dict{Symbol,Float64})
while loading In[4], in expression starting on line 17

We can see that the diode in this case acts like a voltage "limiter" at one end - in the limited region, the diode is acting like a short circuit, plus a constant ~$0.7 \textrm{ V}$ voltage drop. This can be extended to form a useful circuit ...

### Example 3: Voltage limiter

<img src="img/example3.png" />

This works the same way as the above circuit does - but now we have two diodes in parallel, facing opposite directions - so the voltage is limited on both sides. For a more interesting result, let's put two diodes in series in one branch, so that the limit on one side should be twice as large as the other side.

In [5]:
circ = Circuit()
vin = DCVoltageSource(:V)
r1 = Resistor(5e+3)

# the default parameters are reasonable - we could investigate variations in the ideality factor, temperuture,
# saturation current, etc
d1 = Diode()
d2 = Diode()
d3 = Diode()

connect!(circ, vin.pLow, circ.gnd)
connect!(circ, vin.pHigh, r1.p1)

# connect the first (single) diode branch
connect!(circ, r1.p2, d1.pIn)
connect!(circ, d1.pOut, circ.gnd)

# connect the second (double) diode branch
connect!(circ, r1.p2, d2.pOut)
connect!(circ, d2.pIn, d3.pOut)
connect!(circ, d3.pIn, circ.gnd)

V_range = -3.:0.01:3.
cop = dc_sweep(circ, :V, V_range, params)

# plot the voltage on the diode
grid(b=true, which="major")
title("Voltage response of the diode circuit")
xlabel("Vin")
ylabel("Vout")
plot(V_range, cop[r1.p2]);

LoadError: LoadError: MethodError: `dc_sweep` has no method matching dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::FloatRange{Float64}, ::Dict{Symbol,Union{Array{Float64,1},Float64}})
Closest candidates are:
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any)
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any, !Matched::Dict{Symbol,Float64})
while loading In[5], in expression starting on line 24

Let's run transient analysis on the above circuit, with a sinusoidal input (note that because there's no components that introduce time derivatives, this is equivalent to a DC sweep):

In [6]:
# we have to replace the DCVoltageSource with a VoltageSource - probably a poor design decision there
# disconnect both it's ports
methods(disconnect!)
disconnect!(circ, vin.pLow)
disconnect!(circ, vin.pHigh)

# replace it
vin = VoltageSource(:(1.3*sin(2pi * t)))
connect!(circ, vin.pLow, circ.gnd)
connect!(circ, vin.pHigh, r1.p1)

# do the analysis
trange = 0.:0.01:6
cop = trans(circ, trange)

grid(b=true, which="major")
title("Voltage limiter response to a sinusoidal input")
xlabel("time (seconds)")
ylabel("Voltage (V)")
plot(trange, cop[vin.pHigh])
plot(trange, cop[r1.p2])
legend(["Input voltage", "Output voltage"]);

LoadError: LoadError: MethodError: `newton` has no method matching newton(::Function, ::Function, ::Array{Float64,1}, ::Dict{Symbol,Float64})
Closest candidates are:
  newton(::Function, ::Function, ::Array{Float64,1})
  newton(::Function, ::Function, ::Array{Float64,1}, !Matched::Dict{Symbol,Union{Array{Float64,1},Float64}})
while loading In[6], in expression starting on line 14

### Example 4: Low pass voltage limiter

Now we can extend the above circuit to one that has state by adding a capacitor in parallel with the diodes, forming a low pass filter.

In [7]:
c1 = Capacitor(3e-7)

# connect it across the diode branches
connect!(circ, c1.p1, r1.p2)
connect!(circ, c1.p2, circ.gnd)

# set the input voltage frequency to 700 Hz
f = 0.7e+3
vin.V = :(1.5*sin(2pi*$(f)*t))

# set a small range as appropriate for a high frequency
trange = 0.:0.00001:0.03
cop = trans(circ, trange)

grid(b=true, which="major")
plot(trange, cop[r1.p2])

# now do it again, for a lower frequency
f = 0.1e+3
vin.V = :(1.3*sin(2pi*$(f)*t))

cop = trans(circ, trange)
plot(trange, cop[r1.p2])

legend(["700 Hz response", "100 Hz response"])
xlabel("time (s)")
ylabel("Output voltage (V)");

LoadError: LoadError: MethodError: `newton` has no method matching newton(::Function, ::Function, ::Array{Float64,1}, ::Dict{Symbol,Float64})
Closest candidates are:
  newton(::Function, ::Function, ::Array{Float64,1})
  newton(::Function, ::Function, ::Array{Float64,1}, !Matched::Dict{Symbol,Union{Array{Float64,1},Float64}})
while loading In[7], in expression starting on line 13

### Example 5: Simple NPN BJT circuit

In [8]:
circ = Circuit()
vcc = DCVoltageSource(10.)
rc = Resistor(100.)
rb1 = Resistor(30e+3)
rb2 = Resistor(10e+3)
q1 = NPN(βf=100.)

connect!(circ, vcc.pLow, circ.gnd)
connect!(circ, vcc.pHigh, rc.p1, "VCC")
connect!(circ, rc.p2, q1.pC, "Vout")
connect!(circ, rb1.p1, vcc.pHigh)
connect!(circ, rb1.p2, q1.pB, "VB")
connect!(circ, rb2.p1, rb1.p2)
connect!(circ, rb2.p2, circ.gnd)
connect!(circ, q1.pE, circ.gnd)

cop = op(circ)

LoadError: LoadError: MethodError: `newton` has no method matching newton(::Function, ::Function, ::Array{Float64,1}, ::Dict{Symbol,Float64})
Closest candidates are:
  newton(::Function, ::Function, ::Array{Float64,1})
  newton(::Function, ::Function, ::Array{Float64,1}, !Matched::Dict{Symbol,Union{Array{Float64,1},Float64}})
while loading In[8], in expression starting on line 17

What happens to the circuit as $\beta_f$, the ratio between collector and base currents in forward active mode, varies?

In [9]:
# what happens to the node voltages as the forward Ic/Ib ratio parameter (βf) varies?
q1.βf = :βf

params = Parameters(:βf => 50.)

βf_range = 50.:1.:500.
cop = dc_sweep(circ, :βf, βf_range, params)

# plot the output voltage, and also plot the current through VCC
subplot(2, 1, 1)
grid(b=true, which="major")
title("Collector voltage (V) vs forward beta")
plot(βf_range, cop[q1.pC])
subplot(2, 1, 2)
title("Supply current (mA) vs forward beta")
grid(b=true, which="major")
plot(βf_range, 1e+3 * cop[vcc]);

LoadError: LoadError: MethodError: `dc_sweep` has no method matching dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::FloatRange{Float64}, ::Dict{Symbol,Union{Array{Float64,1},Float64}})
Closest candidates are:
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any)
  dc_sweep(::SimpleCircuits.Circuit, ::Symbol, ::Any, !Matched::Dict{Symbol,Float64})
while loading In[9], in expression starting on line 7

The transistor is in the forward active region up until $\beta_f = 400$, beyond which the transistor is saturated.

### Example 6: NPN BJT circuit with coupling capacitors and small signal input

In [10]:
# first, change βf back to a number
q1.βf = 100.

# use the previous circuit, and add in the following:
#  - a capacitively coupled input signal on the base
#  - a capacitively coupled load on the collector

# input coupling capacitor
cc_in = Capacitor(1e-6)
cc_out = Capacitor(1e-6)

# input source
f = 200.
vin = VoltageSource(:(0.04*sin(2π*$(f)*t)))

# output load
rl = Resistor(10e+3)

# connect the input
# add a small resistor here, for stability
rs = Resistor(100.)
connect!(circ, cc_in.p2, q1.pB)
connect!(circ, cc_in.p1, rs.p2)
connect!(circ, vin.pHigh, rs.p1)
connect!(circ, vin.pLow, circ.gnd)

# connect the output
#connect!(circ, cc_out.p1, q1.pC)
#connect!(circ, cc_out.p2, rl.p1, "Vload")
#connect!(circ, rl.p2, circ.gnd);

In [11]:
f = 200.
vin.V = :(0.00001*sin(2π*$(f)*t))

cc_in.C = 1e-6

# run transient analysis
trange = 0.:0.000004:0.15
cop_trans = trans(circ, trange)

# this is obviously not correct
#plot(trange, cop_trans[rl.p1])
plot(trange, cop_trans[vin.pHigh])
plot(trange, cop_trans[q1.pB])
#plot(trange, cop_trans[q1.pC])

# my GUESS is that this is happening because we've provided the initial time derivative as the zero vector
# but this is not necessarily true

LoadError: LoadError: MethodError: `newton` has no method matching newton(::Function, ::Function, ::Array{Float64,1}, ::Dict{Symbol,Float64})
Closest candidates are:
  newton(::Function, ::Function, ::Array{Float64,1})
  newton(::Function, ::Function, ::Array{Float64,1}, !Matched::Dict{Symbol,Union{Array{Float64,1},Float64}})
while loading In[11], in expression starting on line 8

## Part 3: Further Work

### General interface cleanup/organization

The idea of this project is to implement a programmable circuit simulator that's easy and intuitive to use - there's a lot of room for improvement here. For example, the 

### Improvements to the way 'parameters' are handled

### Improvements to the Newton-Raphson implementation

### Improvements to transient analysis

### Implement MOSFETS

### 